In [5]:
import tensorflow as tf
import tf_retinanet
import os

In [6]:
dataset = tf_retinanet.data.read(os.path.join('data', 'headset.record-?????-of-00010'))

In [9]:
for image, target in dataset.take(1):
    print(image)

tf.Tensor(
[[[[  9.061        8.221        5.32      ]
   [  9.324125     8.484125     5.583125  ]
   [  9.832875     8.992875     6.091875  ]
   ...
   [127.061      118.221      105.32      ]
   [127.061      118.221      105.32      ]
   [127.061      118.221      105.32      ]]

  [[  9.324125     8.484125     5.583125  ]
   [  9.51801523   8.67801523   5.77701523]
   [  9.89290039   9.05290039   6.15190039]
   ...
   [127.061      118.221      105.32      ]
   [127.061      118.221      105.32      ]
   [127.061      118.221      105.32      ]]

  [[  9.832875     8.992875     6.091875  ]
   [  9.89290039   9.05290039   6.15190039]
   [ 10.00895898   9.16895898   6.26795898]
   ...
   [127.061      118.221      105.32      ]
   [127.061      118.221      105.32      ]
   [127.061      118.221      105.32      ]]

  ...

  [[ 57.201625    47.1335      43.548125  ]
   [ 56.35222461  46.870375    42.55565039]
   [ 54.70991602  46.361625    40.63670898]
   ...
   [-83.24575195 -91.085

In [ ]:
def batch_modify(image, target):
    print(tf.shape(image), target)
    return image, target
modified_dataset = dataset.batch(4).map(batch_modify)